In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# data manipulation
import pandas as pd
import numpy as np

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# datetime manipulations
from datetime import datetime
import itertools as it

# modeling
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AR
from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics

import math

# supporting modules
import summarize
import acquire
import prepare

In [6]:
pd.set_option('display.max_rows', None)

In [7]:
df = acquire.acquire_data()

In [8]:
df = prepare.prepare_data(df)

In [10]:
df.shape

(225, 11)

Time to Impute!

In [9]:
df.resample('D').mean()

,calories_burned,steps,distance,floors,minutes_sedentary,minutes_lightly_active,minutes_fairly_active,minutes_very_active,activity_calories,total_minutes_logged,distance_per_step
date,,,,,,,,,,,
2018-04-26,2635,2931,1.38,2,1347,93,0,0,566,1440,0.000471
2018-04-27,3649,9971,4.70,19,1158,219,46,17,1752,1440,0.000471
2018-04-28,3379,3521,1.66,2,702,155,22,11,1124,890,0.000471
2018-04-29,2851,1639,0.77,0,891,122,0,0,602,1013,0.000470
2018-04-30,2825,1323,0.62,0,880,104,0,0,540,984,0.000469
2018-05-01,3292,2581,1.22,0,919,167,13,2,1068,1101,0.000473
2018-05-02,2660,1372,0.65,0,927,90,0,0,419,1017,0.000474
2018-05-03,3121,2232,1.05,0,876,157,16,0,1006,1049,0.000470
2018-05-04,2522,904,0.43,1,1012,61,0,0,269,1073,0.000476


In [11]:
df['calories_burned'].replace(to_replace= 0, value= 3317, inplace=True)

In [12]:
df['steps'].replace(to_replace= 0, value= 6916, inplace=True)

In [13]:
df['distance'].replace(to_replace= 0, value= 3, inplace=True)

In [14]:
df['floors'].replace(to_replace= 0, value= 6, inplace=True)

In [15]:
df['minutes_sedentary'].replace(to_replace= 0, value= 920, inplace=True)

In [16]:
df['minutes_lightly_active'].replace(to_replace= 0, value= 172, inplace=True)

In [17]:
df['minutes_fairly_active'].replace(to_replace= 0, value= 17, inplace=True)

In [18]:
df['minutes_very_active'].replace(to_replace= 0, value= 22, inplace=True)

In [20]:
df['activity_calories'].replace(to_replace= 0, value= 1331, inplace=True)

In [21]:
df['total_minutes_logged'].replace(to_replace= 0, value= 1131, inplace=True)

In [22]:
df['distance_per_step'].replace(to_replace= 0, value= .000472, inplace=True)

In [23]:
df

,calories_burned,steps,distance,floors,minutes_sedentary,minutes_lightly_active,minutes_fairly_active,minutes_very_active,activity_calories,total_minutes_logged,distance_per_step
date,,,,,,,,,,,
2018-04-26,2635,2931,1.38,2,1347,93,17,22,566,1440,0.000471
2018-04-27,3649,9971,4.70,19,1158,219,46,17,1752,1440,0.000471
2018-04-28,3379,3521,1.66,2,702,155,22,11,1124,890,0.000471
2018-04-29,2851,1639,0.77,6,891,122,17,22,602,1013,0.000470
2018-04-30,2825,1323,0.62,6,880,104,17,22,540,984,0.000469
2018-05-01,3292,2581,1.22,6,919,167,13,2,1068,1101,0.000473
2018-05-02,2660,1372,0.65,6,927,90,17,22,419,1017,0.000474
2018-05-03,3121,2232,1.05,6,876,157,16,22,1006,1049,0.000470
2018-05-04,2522,904,0.43,1,1012,61,17,22,269,1073,0.000476
